# viz

> Visulaization modules

In [ ]:
#| default_exp viz

In [ ]:
#| hide
from nbdev.showdoc import *

In [ ]:
#| export

import numpy as np
import os
import matplotlib as mpl
import matplotlib.pylab as plt
import pandas as pd

In [ ]:
#| export

def plot_lines_with_param_color(param_array:np.array=None, # parameter array
                                x_array:np.array=None, # x-axis array
                                y_array_all:np.array=None, # y-axis array
                                title_str:str=None, # Title string
                                xlabel_str:str=None, # x-label string
                                ylabel_str:str=None, # y-label string
                                param_name_str:str=None, # Parameter string
                               ):
    
    
    f = plt.figure(32, figsize=(10, 5))    

    norm = mpl.colors.Normalize(vmin=param_array.min(), vmax=param_array.max())
    cmap = mpl.cm.ScalarMappable(norm=norm, cmap=mpl.cm.plasma)
    cmap.set_array([])

    for sim_index in range(50):
        plt.plot(x_array, y_array_all[sim_index], 
                 '-', alpha= 0.5, c=cmap.to_rgba(param_array[sim_index]), 
                 label='Sim: '+str(sim_index) )

    # plt.plot(stellar_mass_test, gsmf_um_test, 'k.', label='UM z=0.00')
    plt.xscale('log')
    # plt.axhline(y=0, linestyle='dashed', color='black')
    # plt.yscale('log')
    # plt.xlim(4e9, )
    # plt.ylim(-0.02, 0.02)

    clb = plt.colorbar(cmap)

    clb.ax.tick_params(labelsize=15) 
    clb.ax.set_title(param_name_str, fontsize=15)

    plt.title(title_str, fontsize=15)
    plt.xlabel(xlabel_str, fontsize=15)
    plt.ylabel(ylabel_str, fontsize=15)
    
    return f

In [ ]:
#| export

def plot_scatter_matrix(df, 
                        colors
                       ): 
    
    f, a = plt.subplots(1,1, figsize = (10, 10))
    scatter_matrix = pd.plotting.scatter_matrix(df, 
                                                color=colors,  
                                                figsize=(10,10), 
                                                alpha=1.0, 
                                                ax=a, 
                                                grid=False, 
                                                diagonal='hist',
                                                range_padding=0.1,
                                                s=80);

    for ax in scatter_matrix.ravel():
        ax.set_xlabel(ax.get_xlabel(), fontsize = 14, rotation = 0)
        ax.set_ylabel(ax.get_ylabel(), fontsize = 14, rotation = 90)
        
    return f

In [ ]:
#| hide
import nbdev; nbdev.nbdev_export()

/home/nramachandra/anaconda3/envs/qom_torch_tf/lib/python3.9/site-packages/nbdev/export.py:54: UserWarning: Notebook '/home/nramachandra/Projects/MG_emu/Codes/CubicGalileonEmu/nbs/index.ipynb' uses `#|export` without `#|default_exp` cell.
Note nbdev2 no longer supports nbdev1 syntax. Run `nbdev_migrate` to upgrade.
See https://nbdev.fast.ai/getting_started.html for more information.
  warn(f"Notebook '{nbname}' uses `#|export` without `#|default_exp` cell.\n"
